In [ ]:
pip install requests

In [ ]:
pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 7.4 MB/s eta 0:00:00


In [ ]:
pip install docutils

In [ ]:
pip install awscli

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 kB 33.0 MB/s eta 0:00:00
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9
    Uninstalling rsa-4.9:
      Successfully uninstalled rsa-4.9
  Attempting uninstall: docutils
    Found existing installation: docutils 0.21.2
    Uninstalling docutils-0.21.2:
      Successfully uninstalled docutils-0.21.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx 8.1.3 requires docutils<0.22,>=0.20, but you have docutils 0.16 which is incompatible.


In [ ]:
import requests
import json
import logging

# Konfigurasi Cloudflare API
CLOUDFLARE_API_TOKEN = "4NwO0UP_T0aVtS8XzzkJnePwpRTtZtphkfJkLfLK"
CLOUDFLARE_ZONE_ID = "e430952b40683784f19e1d3e0bc16566"
DNS_RECORD_NAME = "laboratoriumweb.id"  # Nama DNS yang ingin di-update

# Konfigurasi Logging
LOG_FILE = "dns_update.log"
logging.basicConfig(
    filename=LOG_FILE,
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

# Fungsi untuk mendapatkan IP publik
def get_public_ip():
    try:
        response = requests.get("https://api.ipify.org?format=json")
        response.raise_for_status()
        return response.json()["ip"]
    except Exception as e:
        logging.error(f"Gagal mendapatkan IP publik: {e}")
        return None

# Fungsi untuk mendapatkan detail DNS record
def get_dns_record():
    headers = {"Authorization": f"Bearer {CLOUDFLARE_API_TOKEN}"}
    url = f"https://api.cloudflare.com/client/v4/zones/{CLOUDFLARE_ZONE_ID}/dns_records?name={DNS_RECORD_NAME}"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        records = response.json()["result"]
        return records[0] if records else None
    else:
        logging.error(f"Error mendapatkan DNS record: {response.status_code} - {response.text}")
        return None

# Fungsi untuk memperbarui DNS record
def update_dns_record(record_id, ip):
    headers = {
        "Authorization": f"Bearer {CLOUDFLARE_API_TOKEN}",
        "Content-Type": "application/json",
    }
    url = f"https://api.cloudflare.com/client/v4/zones/{CLOUDFLARE_ZONE_ID}/dns_records/{record_id}"
    data = {
        "type": "A",  # Tipe record, sesuaikan jika menggunakan IPv6 ("AAAA")
        "name": DNS_RECORD_NAME,
        "content": ip,
        "ttl": 1,  # TTL 1 artinya auto
        "proxied": True,  # Set True jika menggunakan Cloudflare proxy
    }
    response = requests.put(url, headers=headers, json=data)
    if response.status_code == 200:
        logging.info(f"DNS record diperbarui ke IP: {ip}")
        print("DNS record berhasil diperbarui.")
    else:
        logging.error(f"Error memperbarui DNS record: {response.status_code} - {response.text}")
        print(f"Error memperbarui DNS record: {response.status_code} - {response.text}")

# Main script
def main():
    current_ip = get_public_ip()
    if not current_ip:
        print("Tidak dapat melanjutkan tanpa IP publik.")
        return

    print(f"IP publik saat ini: {current_ip}")

    dns_record = get_dns_record()
    if dns_record:
        record_id = dns_record["id"]
        current_dns_ip = dns_record["content"]
        if current_dns_ip != current_ip:
            logging.info(f"IP berubah dari {current_dns_ip} ke {current_ip}. Memperbarui DNS record.")
            print(f"IP berbeda. Memperbarui dari {current_dns_ip} ke {current_ip}.")
            update_dns_record(record_id, current_ip)
        else:
            logging.info("IP sudah sesuai. Tidak ada pembaruan yang diperlukan.")
            print("IP sudah sesuai. Tidak perlu pembaruan.")
    else:
        logging.error("DNS record tidak ditemukan.")
        print("DNS record tidak ditemukan.")

if __name__ == "__main__":
    main()

IP publik saat ini: 34.16.190.190
IP berbeda. Memperbarui dari 34.13.133.143 ke 34.16.190.190.
DNS record berhasil diperbarui.
